# CS-GY 9223-D: Deep Learning Homework 2
Due on Friday, 12th March 2019, 11:55 PM

This homework can be done in pairs.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Hupo Tang, ht1073

Member 2: Name, NetID

In [1]:
import os
import numpy as np
import glob
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD, RMSprop, Adam

# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, Lambda, Reshape, ZeroPadding2D
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.utils import to_categorical

import cv2
# import matplotlib.pylab as plt
# %matplotlib inline

Using TensorFlow backend.


In [9]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [3]:
data_path = './'
TRAIN = './images_training_revtest/'
TEST = './images_test_rev1/'
LABELS = './training_solutions_rev1.csv'

In [31]:
g = glob.glob(data_path+'images_training_revtest/*.jpg')
shuf = np.random.permutation(g)
for i in range(10):
    os.rename(shuf[i], data_path+ 'valid/' + shuf[i].split("/")[-1])

In [145]:
from random import shuffle
from scipy.misc import imresize
import csv

class data_loader:    
    """
    Creates a class for handling train/valid/test data paths,
    training labels and image IDs.
    Useful for switching between sample and full datasets.
    """
    def __init__(self, path):    
        self.path = path 
        self.train_path = TRAIN
        #self.val_path = path + "valid"
        self.test_path = TEST
        
        def get_paths(directory):
            return [f for f in os.listdir(directory)]
        
        self.training_images_paths = get_paths(self.train_path)
        # self.validation_images_paths = get_paths(self.val_path)
        self.test_images_paths = get_paths(self.test_path)    
        
        def get_all_solutions():
        # Import solutions file and load into self.solutions
            all_solutions = {}
            # /'training_solutions_rev1.csv'
            with open(LABELS, 'r') as f:
                reader = csv.reader(f, delimiter=",")
                next(reader)
                for i, line in enumerate(reader):
                    all_solutions[line[0]] = [float(x) for x in line[1:]]
            return all_solutions
        
        self.all_solutions = get_all_solutions()

    def get_id(self,fname):
        return fname.replace(".jpg","").replace("data","")
        
    def find_label(self,val):
        return self.all_solutions[val]

def rotate90(img):
    return np.rot90(img,axes=(1,2))

def rotate180(img):
    tmp = np.rot90(img,axes=(1,2))
    return np.rot90(tmp,axes=(1,2))

def rotate270(img):
    tmp = np.rot90(img,axes=(1,2))
    tmp = np.rot90(tmp,axes=(1,2))
    return np.rot90(tmp,axes=(1,2))

def flip_h(img):
    return np.flip(img, axis=1)

def flip_v(img):
    return np.flip(img, axis=2)

def process_images(paths,aug=0):
    """
    Import image at 'paths', decode, centre crop and prepare for batching. 
    """
    count = len(paths)
#     print(count)
    arr = np.zeros(shape=(count,3,69,69))
    for c, path in enumerate(paths):
        raw_img = cv2.imread(path).T  # 3,424,424
#         print(raw_img.shape)
#         img = raw_img
        if not aug:
            img=raw_img
        else:
            auger={1:rotate90,2:rotate180,3:rotate270,4:flip_h,5:flip_v}.get(aug)
            img=auger(raw_img)
        img = img[:,108:315,108:315] #crop 424x424 -> 207x207
        img = imresize(img,size=(69,69,3),interp="cubic").T # downsample to half res
        
#         print(img.shape)
        img1 = np.sqrt(img[:1,:,:]*255)
        img2 = np.sqrt(img[1:2,:,:])
        img2 = np.append(img1, img2, axis=0)
        img3 = np.sqrt(img[2:3,:,:])
        img = np.append(img2, img3.reshape((1,69,69)), axis=0)
#         print(img.shape)
        
        arr[c] = img
#     print(arr.shape)
    return arr


def BatchGenerator(getter):
    while 1:
        for f in getter.training_images_paths:
            for aug in {0,1,2,3,4,5}:
                X_train = process_images([getter.train_path + '/' + fname for fname in [f]], aug)
                X_train = np.reshape(X_train,(1,69,69,3))
                id_ = getter.get_id(f)
                y_train = np.array(getter.find_label(id_))
                y_train = np.reshape(y_train,(1,37))
                assert(X_train.shape==(1,69,69,3))
                yield (X_train, y_train)


In [123]:
fetcher = data_loader(data_path)
# fetcher.training_images_paths

In [83]:
# VGG_16 Network

def ConvBlock(layers, model, filters):
    """
    Create a layered Conv/Pooling block
    """
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))  # zero padding of size 1
        model.add(Convolution2D(filters, 3, 3, activation='relu'))  # 3x3 filter size 
    model.add(MaxPooling2D((1,1), strides=(2,2)))

def FCBlock(model):
    """
    Fully connected block with ReLU and dropout
    """
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG_16():
    """
    Implement VGG16 architecture
    """
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(3,69,69)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    
    model.add(Dense(37, activation = 'sigmoid'))
    return model


In [62]:
# Compile 
optimizer = RMSprop(lr=1e-6)
model = VGG_16()
model.compile(loss='mean_squared_error', optimizer=optimizer)

(None, 2, 35, 64)
(None, 1, 5, 512)


In [141]:
def MyCNN():
    model = Sequential()
    #NHWC(,69,69,3)
    model.add(Lambda(lambda x: x, input_shape=(69,69,3)))
#     print(model.output_shape)    
#     model.add(ZeroPadding2D((2,2)))
    model.add(Convolution2D(32, 6, 6, activation='relu'))

    model.add(MaxPooling2D((2,2), strides=(2,2)))
#     model.add(ZeroPadding2D((2,2)))
    model.add(Convolution2D(64, 5, 5, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(37, activation='softmax'))
    return model

In [142]:
# Compile 
# optimizer = RMSprop(lr=1e-5)
myModel = MyCNN()
myModel.compile(loss='mean_squared_error', optimizer=optimizer)

In [70]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

In [71]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath='./weights.hdf5', verbose=1, save_best_only=True)

batch_size = 32
steps_to_take = int(len(fetcher.training_images_paths)*6/batch_size)
#val_steps_to_take = int(len(fetcher.validation_images_paths)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

#model = load_model('tmp/weights.hdf5')

history = LossHistory()

In [27]:
hist = model.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=5,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )

Epoch 1/5
 - 20s - loss: 0.1813
Epoch 2/5
 - 21s - loss: 0.1919
Epoch 3/5
 - 22s - loss: 0.1780
Epoch 4/5
 - 24s - loss: 0.1777
Epoch 5/5
 - 23s - loss: 0.1634


In [146]:
hist2 = myModel.fit_generator(BatchGenerator(fetcher),
                    samples_per_epoch=steps_to_take, 
                    nb_epoch=20,
                    verbose=2,
                    callbacks=[history,checkpointer,early_stopping],
                   )

Epoch 1/20
 - 26s - loss: 0.0651
Epoch 2/20
 - 19s - loss: 0.0513
Epoch 3/20
 - 16s - loss: 0.0633
Epoch 4/20
 - 16s - loss: 0.0653
Epoch 5/20
 - 16s - loss: 0.0507
Epoch 6/20
 - 16s - loss: 0.0634
Epoch 7/20
 - 16s - loss: 0.0640
Epoch 8/20
 - 16s - loss: 0.0486
Epoch 9/20
 - 16s - loss: 0.0626
Epoch 10/20
 - 17s - loss: 0.0503
Epoch 11/20
 - 16s - loss: 0.0528
Epoch 12/20
 - 17s - loss: 0.0472
Epoch 13/20
 - 16s - loss: 0.0489
Epoch 14/20
 - 15s - loss: 0.0709
Epoch 15/20
 - 16s - loss: 0.0518
Epoch 16/20
 - 16s - loss: 0.0646
Epoch 17/20
 - 16s - loss: 0.0464
Epoch 18/20
 - 16s - loss: 0.0595
Epoch 19/20
 - 15s - loss: 0.0551
Epoch 20/20
 - 16s - loss: 0.0554


In [96]:
print(steps_to_take)
hist2 = myModel.fit_generator(BatchGenerator(fetcher),
          samples_per_epoch=steps_to_take,
          epochs=5,
          verbose=2,
          callbacks=[history,checkpointer,early_stopping],
         )

127
Epoch 1/5
 - 27s - loss: 0.0943
Epoch 2/5
 - 22s - loss: 0.0899
Epoch 3/5
 - 21s - loss: 0.0900
Epoch 4/5
 - 23s - loss: 0.0871
Epoch 5/5
 - 22s - loss: 0.0832


In [89]:
model.save_weights('./weights.hdf5')

In [147]:
11*10//9+8-7*6//5+4-3*2//1

10